In [ ]:
import pprint
import boto3

pp = pprint.PrettyPrinter(indent=4)

client = boto3.client("s3")
buckets = client.list_buckets()
    
for bucket in buckets['Buckets']:
    print('Name: ' + bucket['Name'])



In [ ]:

# pagination example
cwl = boto3.client("logs")

paginator = cwl.get_paginator('describe_log_groups')
page_iterator = paginator.paginate(logGroupNamePrefix="/aws",
PaginationConfig={
        'MaxItems': 10000,
        'PageSize': 20
    })

page_iterations_count=0
log_groups_count=0
for page in page_iterator:
    page_iterations_count+=1
    log_groups_count += len(page['logGroups'])

print(f"page_iterations_count: {page_iterations_count}, log_groups_count: {log_groups_count}")
    



In [ ]:
# AWS Step Functions Data Science SDK Example
# https://aws-step-functions-data-science-sdk.readthedocs.io/

import stepfunctions
import logging

from stepfunctions.steps import *
from stepfunctions.workflow import Workflow

stepfunctions.set_stream_logger(level=logging.INFO)

workflow_execution_role = "arn:aws:iam::529276214230:role/StepFunctionsWorkflowExecutionRole" # paste the StepFunctionsWorkflowExecutionRole ARN from above

start_pass_state = Pass(
    state_id="MyPassState"             
)

# First we chain the start pass state
basic_path=Chain([start_pass_state])

basic_workflow = Workflow(
    name="MyWorkflow_Simple",
    definition=basic_path,
    role=workflow_execution_role
)

#Render the workflow
basic_workflow.render_graph()

print(basic_workflow.definition.to_json(pretty=True))


In [8]:
basic_workflow.create()

basic_workflow_execution = basic_workflow.execute()

basic_workflow_execution.render_progress()

basic_workflow_execution.list_events(html=True)

# generates an AWS CloudFormation Template to deploy your workflow.
basic_workflow.get_cloudformation_template()

[WARNING] To reuse the CloudFormation template in different regions, please make sure to update the region specific AWS resources in the StateMachine definition.


'AWSTemplateFormatVersion: \'2010-09-09\'\nDescription: CloudFormation template for AWS Step Functions - State Machine\nResources:\n  StateMachineComponent:\n    Type: AWS::StepFunctions::StateMachine\n    Properties:\n      StateMachineName: MyWorkflow_Simple\n      DefinitionString: |-\n        {\n          "StartAt": "MyPassState",\n          "States": {\n            "MyPassState": {\n              "Type": "Pass",\n              "End": true\n            }\n          }\n        }\n      RoleArn: arn:aws:iam::529276214230:role/StepFunctionsWorkflowExecutionRole\n'